In [1]:
import pandas as pd
import numpy as np
import dash
import flask
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from pandas_datareader import data as web
from datetime import datetime as dt

In [2]:
import pymongo

In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
password = os.getenv("PASSWORD")
username = os.getenv("USERNAME")
location = os.getenv("LOCATION")
collection = os.getenv("COLLECTION")

In [4]:
location

'27017/1grams'

In [5]:
collection

'en'

In [6]:
client = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client[location[6:]]

In [7]:
def load(word):
    try:
        df = pd.DataFrame(list(db[collection].find({ "word": word})))
        df = df.sort_values(by=['time'])
        df['year'] = [date.year for date in df['time']]
        df['day'] = [date.timetuple().tm_yday for date in df['time']]
        return df
    except:
        print(str("No timeseries found for "+word))
        df = pd.DataFrame()
        return df

In [ ]:
q = load('@realdonaldtrump')
q

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

# Input box for query

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

data=[]

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    dcc.Input(id='my-id', value='@realdonaldtrump', type='text'),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}}),
])

@app.callback(Output('my-graph', 'figure'), [Input('my-id', 'value')])
def update_graph(input_value):
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    input_value = input_value.replace({"   ":",","  ":","," ":",",",,":",",",,,":","})
    values = input_value.split(',')
    for item in values:
        item = lower(item)
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure

if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

# Dropdown for query

### Load wordlist

In [ ]:
import csv
wordlist = []
with open('wordlist_from-df.csv', 'rt') as f:
    reader = csv.reader(f)
    next(reader)
    wordlist=list(reader)

In [ ]:
wordDF=pd.DataFrame(wordlist, columns=['word','rank'])

In [ ]:
wordDF=wordDF[wordDF['rank']!='Rank']

In [ ]:
wordDF=wordDF[wordDF['rank']!='minRank']

In [ ]:
wordDF['rank']=wordDF['rank'].astype(int);

### Dropdown with under 5k

In [ ]:
wordDF[wordDF['rank']<5000].shape

In [ ]:
options = []
wordDF_filtered = wordDF[wordDF['rank']<5000]
for word in wordDF_filtered['word']:
    options.append({'label': word,'value': word})

In [ ]:
options[:5]

In [ ]:
options[100:105]

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.P('This shows any one-grams that cracked the top 5k rank at any time (47,844 onegrams)'),
    html.Div([
        dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','@bts_twt'], multi=True)]),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}})
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])

def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    input_value = input_value.replace({"   ":",","  ":","," ":",",",,":",",",,,":","})
    for item in input_value:
        item = lower(item)
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

### Dropdown of under 1k list, with ability to add options

In [ ]:
options = []
wordDF_filtered = wordDF[wordDF['rank']<1000]
wordDF_filtered=wordDF_filtered.sort_values(by='rank')
for word in wordDF_filtered['word']:
    options.append(word)

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.P('Select a one-gram that cracked the top 1k rank at any time (1,056 onegrams)'),
    html.Datalist(id="wordlist", children=[]),
    dcc.Input(
        id='input-field',
        name='symbol',
        list='wordlist',
        placeholder='Enter a value...',
        type='text',
        value='trump'
    ),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}})
])

@app.callback(Output('wordlist', 'children'),[Input('input-field', 'value')])
def callback(item):
    if len(item) < 5:
        wordDF_filtered = wordDF[wordDF['rank']<1000]
    else:
        wordDF_filtered = wordDF[wordDF['word'].str.contains(item)]
    for word in wordDF_filtered:
        options.append(word)
    return options

@app.callback(Output('my-graph', 'figure'), [Input('input-field', 'value')])
def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    input_value = input_value.replace({"   ":",","  ":","," ":",",",,":",",",,,":","})
    values = input_value.split(',')
    for item in values:
        item = lower(item)
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

### Dynamic Dropdown (abandoned)

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.P('Select a one-gram that cracked the top 5k rank at any time (47,844 onegrams)'),
    html.Div([
        dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','@bts_twt'], multi=True)]),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}})
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])

def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    for item in input_value:
        item = lower(item)
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure

@app.callback(Output('my-dropdown', 'options'), [Input('my-dropdown', 'value')])

def update_options(input_value):
    for item in input_value:
        if len(item) < 5:
            wordDF_filtered = wordDF[wordDF['rank']<1000]
        else:
            wordDF_filtered = wordDF[wordDF['word'].str.contains(item)]
        for word in wordDF_filtered:
            options.append({'label': word[0],'value': word[0]})
    return options


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')